<a href="https://colab.research.google.com/github/Uganda-lang/Ganda-TTS/blob/main/Notebooks/tts_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl triton cut_cross_entropy unsloth_zoo -q
!pip install sentencepiece protobuf "datasets>=3.4.1" huggingface_hub hf_transfer -q
!pip install --no-deps unsloth -q
!pip install transformers==4.52.3 -q
!pip install snac -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.4/43.4 MB 21.1 MB/s eta 0:00:00
   ━━━━━━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/72.9 MB 108.3 MB/s eta 0:00:01
ERROR: Operation cancelled by user
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 15.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch

In [2]:
from unsloth import FastLanguageModel
import torch

ModuleNotFoundError: No module named 'bitsandbytes'

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "Uganda-lang/Orpheous-Eng-multi-speaker",
    max_seq_length= 2048, # Choose any for long context!
    dtype = None, # Select None for auto detection
    load_in_4bit = True, # Select True for 4bit which reduces memory usage
    token = "[token]", # Replace with your token
)


In [ ]:
from snac import SNAC

snac_model = SNAC.from_pretrained("hubertsiuzdak/snac_24khz",
                                  token="[token]")
snac_model = snac_model.to("cuda")

In [ ]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

# Moving snac_model cuda to cpu
snac_model.to("cuda")

In [ ]:
import torch
import gc
from IPython.display import display, Audio

def generate_audio_from_prompt(
    prompt: str,
    chosen_voice: str,
    model,
    tokenizer,
    snac_model,
    display_result: bool = True
) -> torch.Tensor:
    """
    Generates audio from a text prompt, with explicit memory cleanup steps.

    Args:
        prompt (str): The text prompt to generate audio for.
        chosen_voice (str): The voice to use for the prompt (e.g., "en_speaker_0").
        model: The pre-loaded main language model.
        tokenizer: The pre-loaded tokenizer for the language model.
        snac_model: The pre-loaded SNAC audio model.
        display_result (bool): If True, prints the prompt and displays an audio player.

    Returns:
        torch.Tensor: The generated audio waveform as a PyTorch tensor on the CPU.
    """

    # --- Helper function for audio decoding ---
    def redistribute_codes(code_list, device):
        """Restructures flat code list and places tensors on the correct device."""
        layer_1, layer_2, layer_3 = [], [], []
        for i in range(len(code_list) // 7):
            base_idx = 7 * i
            layer_1.append(code_list[base_idx])
            layer_2.append(code_list[base_idx + 1] - 4096)
            layer_3.append(code_list[base_idx + 2] - (2 * 4096))
            layer_3.append(code_list[base_idx + 3] - (3 * 4096))
            layer_2.append(code_list[base_idx + 4] - (4 * 4096))
            layer_3.append(code_list[base_idx + 5] - (5 * 4096))
            layer_3.append(code_list[base_idx + 6] - (6 * 4096))

        # Create the tensors directly on the target device.
        codes = [
            torch.tensor(layer_1, device=device, dtype=torch.long).unsqueeze(0),
            torch.tensor(layer_2, device=device, dtype=torch.long).unsqueeze(0),
            torch.tensor(layer_3, device=device, dtype=torch.long).unsqueeze(0)
        ]

        # Now, `codes` are on the GPU, matching the `snac_model`.
        audio_hat = snac_model.decode(codes)
        return audio_hat


    device = model.device

    with torch.no_grad():
        # 1. Format and tokenize prompt
        formatted_prompt = f"{chosen_voice}: {prompt}" if chosen_voice else prompt
        input_ids = tokenizer(formatted_prompt, return_tensors="pt").input_ids

        start_token = torch.tensor([[128259]], dtype=torch.int64)
        end_tokens = torch.tensor([[128009, 128260]], dtype=torch.int64)
        modified_input_ids = torch.cat([start_token, input_ids, end_tokens], dim=1)

        # 2. Generate token IDs with the language model
        generated_ids = model.generate(
            input_ids=modified_input_ids.to(device),
            max_new_tokens=2048,
            do_sample=True,
            temperature=0.6,
            top_p=0.95,
            repetition_penalty=1.1,
            num_return_sequences=1,
            eos_token_id=128258,
            use_cache=True
        )

        # 3. Post-process tokens
        token_to_find = 128257
        token_to_remove = 128258

        token_indices = (generated_ids == token_to_find).nonzero(as_tuple=True)
        if len(token_indices[1]) > 0:
            last_occurrence_idx = token_indices[1][-1].item()
            cropped_tensor = generated_ids[:, last_occurrence_idx + 1:]
        else:
            cropped_tensor = generated_ids

        processed_tensor = cropped_tensor[cropped_tensor != token_to_remove]

        row_length = processed_tensor.size(0)
        new_length = (row_length // 7) * 7
        trimmed_tensor = processed_tensor[:new_length]

        code_list = [t.item() - 128266 for t in trimmed_tensor.cpu()]

        # 4. Decode codes into an audio waveform
        samples_gpu = redistribute_codes(code_list, device=device)

        # Move the final result to CPU before returning
        samples_cpu = samples_gpu.squeeze().cpu()

    # Explicitly delete large intermediate tensors
    del generated_ids
    del cropped_tensor
    del processed_tensor
    del trimmed_tensor
    del code_list
    del samples_gpu
    if 'modified_input_ids' in locals(): del modified_input_ids

    # Force garbage collection and empty CUDA cache
    gc.collect()
    torch.cuda.empty_cache()

    # 5. Display and return the result
    if display_result:
        print(f"Prompt: {prompt}")
        display(Audio(samples_cpu.numpy(), rate=24000))

    return samples_cpu

In [ ]:
my_prompt = "Hello I can speak in English as christopher, one of the voices I can speak."
my_voice = "christopher"

# Call the function to generate and display the audio
generated_audio_tensor = generate_audio_from_prompt(
    prompt=my_prompt,
    chosen_voice=my_voice,
    model=model,
    tokenizer=tokenizer,
    snac_model=snac_model
)

In [ ]:
# Supported voices:
# 'Barbara',
# 'Mary',
#  'Jennifer'
#  'Jessica'
#  'Susan'
#  'James'
#  'Linda'
#  'Patricia'
#  'Elizabeth'
#  'Christopher'

In [ ]:
my_prompt = "Or as barbra, this is one of my female voices. Pretty cool right?."
my_voice = "barbara"

# Call the function to generate and display the audio
generate_audio_from_prompt(
    prompt=my_prompt,
    chosen_voice=my_voice,
    model=model,
    tokenizer=tokenizer,
    snac_model=snac_model
)

In [ ]:
my_prompt = "I can also speak as Mary as well."
my_voice = "mary"

# Call the function to generate and display the audio
generate_audio_from_prompt(
    prompt=my_prompt,
    chosen_voice=my_voice,
    model=model,
    tokenizer=tokenizer,
    snac_model=snac_model
)

In [ ]:
my_prompt = "Or I can speak as james as you can see."
my_voice = "James"

# Call the function to generate and display the audio
generate_audio_from_prompt(
    prompt=my_prompt,
    chosen_voice=my_voice,
    model=model,
    tokenizer=tokenizer,
    snac_model=snac_model
)

In [ ]:
my_prompt = "This is my other voice called jessica, I have more voices of jennifer, suzan, linda, patricia and elizabeth. But I will be sharing these voices once I am fully done from baking."
my_voice = "jessica"

# Call the function to generate and display the audio
generate_audio_from_prompt(
    prompt=my_prompt,
    chosen_voice=my_voice,
    model=model,
    tokenizer=tokenizer,
    snac_model=snac_model
)

In [ ]:
# export model to gguf

# Luganda

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "Uganda-lang/Orpheous-Multi-speaker-Lug",
    max_seq_length= 2048, # Choose any for long context!
    dtype = None, # Select None for auto detection
    load_in_4bit = True, # Select True for 4bit which reduces memory usage
    token = "[token]", # Replace with your token
)


<!-- [('Charles', 3259),
 ('Sandra', 2314),
 ('Christopher', 1255),
 ('Mark', 1123),
 ('Barbara', 1114),
 ('Michelle', 1080),
 ('Karen', 1027),
 ('James', 925),
 ('Margaret', 879),
 ('Daniel', 767)] -->

In [ ]:
# Supported voices:
# Charles
# Sandra
# Christopher
# Mark
# Barbara
# Michelle
# Karen
# James
# Margaret
# Daniel

In [ ]:
my_prompt = "Nsobolla okwo'geranga Christopher nga wowulila kati."
my_voice = "christopher"

# Call the function to generate and display the audio
generate_audio_from_prompt(
    prompt=my_prompt,
    chosen_voice=my_voice,
    model=model,
    tokenizer=tokenizer,
    snac_model=snac_model
)

In [ ]:
my_prompt = "Oba neenjogela nga charles wenti."
my_voice = "charles"

# Call the function to generate and display the audio
generate_audio_from_prompt(
    prompt=my_prompt,
    chosen_voice=my_voice,
    model=model,
    tokenizer=tokenizer,
    snac_model=snac_model
)

In [ ]:
my_prompt = "Nina neddoboozi lya Sandra bweliti."
my_voice = "sandra"

# Call the function to generate and display the audio
generate_audio_from_prompt(
    prompt=my_prompt,
    chosen_voice=my_voice,
    model=model,
    tokenizer=tokenizer,
    snac_model=snac_model
)

In [ ]:
my_prompt = "Nsobolla ogwogella bwenti mulino eddoboozi."
my_voice = "michelle"

# Call the function to generate and display the audio
generate_audio_from_prompt(
    prompt=my_prompt,
    chosen_voice=my_voice,
    model=model,
    tokenizer=tokenizer,
    snac_model=snac_model
)

In [ ]:
my_prompt = "Oba nemulino elye'kisajja nga woowulira."
my_voice = "daniel"

# Call the function to generate and display the audio
generate_audio_from_prompt(
    prompt=my_prompt,
    chosen_voice=my_voice,
    model=model,
    tokenizer=tokenizer,
    snac_model=snac_model
)

In [ ]:
my_prompt = "Ninna amaloboozi amalala naye nja kugalaga nga mazze oku tureyininga."
my_voice = "mark"

# Call the function to generate and display the audio
generate_audio_from_prompt(
    prompt=my_prompt,
    chosen_voice=my_voice,
    model=model,
    tokenizer=tokenizer,
    snac_model=snac_model
)

In [ ]:
my_prompt = "Charlissi yimilila awo."
my_voice = "margaret"

# Call the function to generate and display the audio
generate_audio_from_prompt(
    prompt=my_prompt,
    chosen_voice=my_voice,
    model=model,
    tokenizer=tokenizer,
    snac_model=snac_model
)

# Acholi

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "Uganda-lang/Orpheous-Ach-Multi-speaker",
    max_seq_length= 2048, # Choose any for long context!
    dtype = None, # Select None for auto detection
    load_in_4bit = True, # Select True for 4bit which reduces memory usage
    token = "[token]", # Replace with your token
)

In [ ]:
# Supported voices:
# James
# Barbara
# Mitchelle
# Mark
# Christopher

In [ ]:
my_prompt = "Uganda tye ka keme ki lok me pur."
my_voice = "mark"

# Call the function to generate and display the audio
generate_audio_from_prompt(
    prompt=my_prompt,
    chosen_voice=my_voice,
    model=model,
    tokenizer=tokenizer,
    snac_model=snac_model
)

In [ ]:
my_prompt = "Lupur twero nongo kony ma dit ka gunongo ngec me gengo onyo cango two ma balo jami ma i poto."
my_voice = "barbara"

# Call the function to generate and display the audio
generate_audio_from_prompt(
    prompt=my_prompt,
    chosen_voice=my_voice,
    model=model,
    tokenizer=tokenizer,
    snac_model=snac_model
)

In [ ]:
my_prompt = "Ler ma pe gidodo ma woto ka yenyo cam i dye poto obalo cam weng ma tye i poto."
my_voice = "james"

# Call the function to generate and display the audio
generate_audio_from_prompt(
    prompt=my_prompt,
    chosen_voice=my_voice,
    model=model,
    tokenizer=tokenizer,
    snac_model=snac_model
)

In [ ]:
my_prompt = "Gum madwong me timo biacara tye i te yub ma pe jenge i kom gamente."
my_voice = "mitchelle"

# Call the function to generate and display the audio
generate_audio_from_prompt(
    prompt=my_prompt,
    chosen_voice=my_voice,
    model=model,
    tokenizer=tokenizer,
    snac_model=snac_model
)

# Runkyankole

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "Uganda-lang/Orpheus-nyn-multi-speaker",
    max_seq_length= 2048, # Choose any for long context!
    dtype = None, # Select None for auto detection
    load_in_4bit = True, # Select True for 4bit which reduces memory usage
    token = "[token]", # Replace with your token
)

In [ ]:
my_prompt = "Nimbasa kugamba nka Christopher omwiraka eri."
my_voice = "christopher"

# Call the function to generate and display the audio
generate_audio_from_prompt(
    prompt=my_prompt,
    chosen_voice=my_voice,
    model=model,
    tokenizer=tokenizer,
    snac_model=snac_model
)

In [ ]:
# Supported voices:

# Michelle
# James
# Patricia
# Mark
# Elizabeth
# Charles
# Daniel
# Barbara
# Christopher
# Linda

In [ ]:
my_prompt = "Nimbasa kugamba nka Michelle omwiraka eri."
my_voice = "michelle"

# Call the function to generate and display the audio
generate_audio_from_prompt(
    prompt=my_prompt,
    chosen_voice=my_voice,
    model=model,
    tokenizer=tokenizer,
    snac_model=snac_model
)

In [ ]:
my_prompt = "Uganda eteire amaani aha buhingi n'oburiisa."
my_voice = "james"

# Call the function to generate and display the audio
generate_audio_from_prompt(
    prompt=my_prompt,
    chosen_voice=my_voice,
    model=model,
    tokenizer=tokenizer,
    snac_model=snac_model
)

In [ ]:
my_prompt = "Bimwe ebirikugambwa aha reediyo nibihwera abantu kumanya obutare burungi bw'amasharuura gaabo."
my_voice = "patricia"

# Call the function to generate and display the audio
generate_audio_from_prompt(
    prompt=my_prompt,
    chosen_voice=my_voice,
    model=model,
    tokenizer=tokenizer,
    snac_model=snac_model
)

In [ ]:
my_prompt = "Okukyerererwa kufuuhirira nikyo kirikutokooza ebyokurya ebitwine ebiro ebi."
my_voice = "charles"

# Call the function to generate and display the audio
generate_audio_from_prompt(
    prompt=my_prompt,
    chosen_voice=my_voice,
    model=model,
    tokenizer=tokenizer,
    snac_model=snac_model
)

In [ ]:
my_prompt = "Omu disiturikiti ya Kayunga emisiri erikukira obwngi ekashangwa erimu ebicoori ebiine oburwaire."
my_voice = "elizabeth"

# Call the function to generate and display the audio
generate_audio_from_prompt(
    prompt=my_prompt,
    chosen_voice=my_voice,
    model=model,
    tokenizer=tokenizer,
    snac_model=snac_model
)

# Tesso

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "Uganda-lang/Orpheous-teo-multi-speaker",
    max_seq_length= 2048, # Choose any for long context!
    dtype = None, # Select None for auto detection
    load_in_4bit = True, # Select True for 4bit which reduces memory usage
    token = "[token]", # Replace with your token
)

In [ ]:
my_prompt = "Epedorete akoriok aimedaun ejok kanejaas aicoreta nu itikitikere adeka."
my_voice = "christopher"

# Call the function to generate and display the audio
generate_audio_from_prompt(
    prompt=my_prompt,
    chosen_voice=my_voice,
    model=model,
    tokenizer=tokenizer,
    snac_model=snac_model
)

In [ ]:
# Supported voices:
# Mitchelle
# Barbara
# Jessica
# Christopher
# James
# Daniel
# Charles
# Mark

In [ ]:
my_prompt = "Akoru ikorion luegelegela nes ingarakini itunganan."
my_voice = "jessica"

# Call the function to generate and display the audio
generate_audio_from_prompt(
    prompt=my_prompt,
    chosen_voice=my_voice,
    model=model,
    tokenizer=tokenizer,
    snac_model=snac_model
)

In [ ]:
my_prompt = "Iraasit yen emunaara aticepak ikur enyamitos."
my_voice = "james"

# Call the function to generate and display the audio
generate_audio_from_prompt(
    prompt=my_prompt,
    chosen_voice=my_voice,
    model=model,
    tokenizer=tokenizer,
    snac_model=snac_model
)

In [ ]:
my_prompt = "Aipagisanar nes ewai ecie lo ibwaikinet iboro toma aswam."
my_voice = "daniel"

# Call the function to generate and display the audio
generate_audio_from_prompt(
    prompt=my_prompt,
    chosen_voice=my_voice,
    model=model,
    tokenizer=tokenizer,
    snac_model=snac_model
)

In [ ]:
my_prompt = "Isisianakinete isomeroi kwana asiomak eipone lo isubusaere."
my_voice = "barbara"

# Call the function to generate and display the audio
generate_audio_from_prompt(
    prompt=my_prompt,
    chosen_voice=my_voice,
    model=model,
    tokenizer=tokenizer,
    snac_model=snac_model
)